In [1]:
import numpy as np
import json

from ModularCirc.Solver import Solver

from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel
from ModularCirc.Models.KorakianitisMixedModel_parameters import KorakianitisMixedModel_parameters

In [2]:
time_setup_dict = {
            'name': 'TimeTest',
            'ncycles': 40,
            'tcycle': 1.0,
            'dt': 0.001,
            'export_min': 1
        }

# Initializing the parameter object
parobj = KorakianitisMixedModel_parameters()

# Initializing the model 
model = KorakianitisMixedModel(time_setup_dict=time_setup_dict, 
                               parobj=parobj, 
                               suppress_printing=True)

In [3]:
# Initializing the solver
solver = Solver(model=model)
# Solver is being setup: switching off console printing and setting the solver method to "LSODA"
solver.setup(suppress_output=True, 
             method='LSODA')

# Running the model
solver.solve()

In [4]:
final_cycle_values = {}

# Define the indexes of the equivalent to the last cycles
tind_fin  = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                      stop=model.time_object.n_t)

# From each of the components, retrieve the volume (<V>), pressure (<P_i>) and flow (<Q_i>)
for key, value in model.components.items():
    final_cycle_values[key] = {
        'V': value.V.values[tind_fin].mean(),
        'P_i': value.P_i.values[tind_fin].mean(),
        'Q_i': value.Q_i.values[tind_fin].mean()
    }

# Save the results to a JSON file in the tests/expected_outputs directory
with open('../tests/expected_outputs/KorakianitisMixedModel_expected_output.json', 'w') as f:
    json.dump(final_cycle_values, f, indent=4)

In [5]:
# # Load the previous JSON file for comparison
# with open('../tests/expected_outputs/KorakianitisMixedModel_expected_output.json', 'r') as f:
#     previous_output = json.load(f)

# # Compare the new output with the previous one
# for key in final_cycle_values.keys():
#     for param in ['V', 'P_i', 'Q_i']:
#         new_value = final_cycle_values[key][param]
#         old_value = previous_output[key][param]
#         assert np.isclose(new_value, old_value, rtol=1e-5), f"Mismatch in {key} for {param}: {new_value} vs {old_value}"
# print("All values match within the specified tolerance.")

# # The test is successful if no assertion errors are raised